In [267]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [268]:
import os
import numpy as np
import csv
from tabulate import tabulate
import sys
import requests
import json
import pandas as pd
from datetime import datetime, timedelta

In [269]:
census = pd.read_csv("/content/drive/My Drive/415/datasci 415/countypres_2000-2020.csv")
corgis = pd.read_csv("/content/drive/My Drive/415/datasci 415/county_demographics-corgis.csv")

In [270]:
#columns with no NAs
vec = []
for col in corgis.columns:
    if (corgis[col] == -1).sum() > 0:
        vec.append(col)
columns_to_keep = [col for col in corgis.columns if col not in vec]
corgis = corgis[columns_to_keep]
#corgis

In [271]:
# removing data for votes that aren't county related, ie overseas troops etc
# also removing kansas city missouri, as it was included in a county, and as its own thing
with_others = census[census['county_fips'].notna() & (census['county_fips'] != 2938000)]
#with_others

In [272]:
#Combining the different voting type methods for each county in a year into 1 entry.
all_parties_data = (
    with_others.groupby(['year', 'county_fips', 'party'], as_index=False)
    .agg(
        State=('state_po', 'first'),
        county_name=('county_name', 'first'),
        votecount=('candidatevotes', 'sum'),
        totalvotes=('totalvotes', 'first')
    )
)

all_parties_data_wide = all_parties_data.pivot_table(
    index=['year', 'county_fips'],
    columns='party',
    values='votecount',
    aggfunc='sum'
).reset_index()
final_data = pd.merge(all_parties_data_wide, all_parties_data[['year', 'county_fips', 'State', 'county_name', 'totalvotes']].drop_duplicates(), on=['year', 'county_fips'], how='left')
all_parties_data = final_data
# The result is stored in 'final_data'
#all_parties_data

In [273]:
#props
all_parties_data['prop_democrat'] = all_parties_data['DEMOCRAT'] / all_parties_data['totalvotes']
all_parties_data['prop_republican'] = all_parties_data['REPUBLICAN'] / all_parties_data['totalvotes']
all_parties_data['prop_favored_democrat'] = all_parties_data['prop_democrat'] - all_parties_data['prop_republican']

# create winner column
all_parties_data['winner'] = np.where(all_parties_data['DEMOCRAT'] > all_parties_data['REPUBLICAN'], 'DEMOCRAT', 'REPUBLICAN')

# removing the 5 rows where no votes were cast at all
all_parties_data = all_parties_data[(all_parties_data['DEMOCRAT'] > 0) & (all_parties_data['REPUBLICAN'] > 0)]

In [274]:
# removing parties other than democrats and republicans
two_party = all_parties_data.drop(columns=["LIBERTARIAN", "GREEN", "OTHER"])
#two_party

In [304]:
# pivoting the dataset into being one row for each county
single_row = (
    two_party
    .groupby('county_fips')
    .apply(lambda x: pd.Series({
        'State': x['State'].iloc[0],
        'county_name': x['county_name'].iloc[0],
        'total_votes_2020': x.loc[x['year'] == 2020, 'totalvotes'].iloc[0] if not x[x['year'] == 2020].empty else None,
        'total_votes_2016': x.loc[x['year'] == 2016, 'totalvotes'].iloc[0] if not x[x['year'] == 2016].empty else None,
        'total_votes_2012': x.loc[x['year'] == 2012, 'totalvotes'].iloc[0] if not x[x['year'] == 2012].empty else None,
        'total_votes_2008': x.loc[x['year'] == 2008, 'totalvotes'].iloc[0] if not x[x['year'] == 2008].empty else None,
        'total_votes_2004': x.loc[x['year'] == 2004, 'totalvotes'].iloc[0] if not x[x['year'] == 2004].empty else None,
        'total_votes_2000': x.loc[x['year'] == 2000, 'totalvotes'].iloc[0] if not x[x['year'] == 2000].empty else None,
        'prop_favored_dem_2020': x.loc[x['year'] == 2020, 'prop_favored_democrat'].iloc[0] if not x[x['year'] == 2020].empty else None,
        'prop_favored_dem_2016': x.loc[x['year'] == 2016, 'prop_favored_democrat'].iloc[0] if not x[x['year'] == 2016].empty else None,
        'prop_favored_dem_2012': x.loc[x['year'] == 2012, 'prop_favored_democrat'].iloc[0] if not x[x['year'] == 2012].empty else None,
        'prop_favored_dem_2008': x.loc[x['year'] == 2008, 'prop_favored_democrat'].iloc[0] if not x[x['year'] == 2008].empty else None,
        'prop_favored_dem_2004': x.loc[x['year'] == 2004, 'prop_favored_democrat'].iloc[0] if not x[x['year'] == 2004].empty else None,
        'prop_favored_dem_2000': x.loc[x['year'] == 2000, 'prop_favored_democrat'].iloc[0] if not x[x['year'] == 2000].empty else None
    }))
    .reset_index()
)

# Add the 'County' column with formatting
single_row['County'] = single_row['county_name'].str.title() + " County"
single_row = single_row.dropna()
#single_row

<ipython-input-304-3e8287f14948>:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({


In [315]:
#fixing county name differences between this and the corgis dataset
county_fips_dict = {
    51678: "Lexington city",
    51510: "Alexandria city",
    51520: "Bristol city",
    51530: "Buena Vista city",
    51540: "Charlottesville city",
    51550: "Chesapeake city",
    51570: "Colonial Heights city",
    51600: "Fairfax city",
    51620: "Franklin city",
    51760: "Richmond city",
    51770: "Roanoke city",
    24510: "Baltimore city",
    32510: "Carson City"
}

for fips, county in county_fips_dict.items():
    single_row.loc[single_row['county_fips'] == fips, 'County'] = county


# Louisiana has parishes, not counties
single_row.loc[single_row['State'] == 'LA', 'County'] = (
    single_row.loc[single_row['State'] == 'LA', 'County']
    .str.strip()
    .str.replace(r'\s+County$', ' Parish', regex=True)  # Replace ' County' at the end with ' Parish'
)
single_row['County'] = single_row['County'].str.upper()
corgis['county_names_with_casing'] = corgis['County']
corgis['County'] = corgis['County'].str.upper()

In [316]:
# Find counties in 'corgis' that are not in 'single_row'
missing_counties = corgis[~corgis['County'].isin(single_row['County'])]['County']

# Filter counties that are in 'missing_counties' and state is 'VA'
test = corgis[(corgis['County'].isin(missing_counties)) & (corgis['State'] == 'VA')]['County']

# Modify the 'County' names by removing the last word and appending 'COUNTY'
virginia_city = test.str.replace(r'\s+\w*$', '', regex=True) + " COUNTY"

# Update the 'County' in 'single_row' where 'County' is in 'virginia_city' and 'State' is 'VA'
single_row.loc[(single_row['County'].isin(virginia_city)) & (single_row['State'] == 'VA'), 'County'] = test.values


In [317]:
single_row.loc[single_row['county_fips'] == 11001, 'County'] = "DISTRICT OF COLUMBIA"
single_row.loc[single_row['county_fips'] == 29510, 'County'] = "ST. LOUIS CITY"
single_row.loc[single_row['county_fips'] == 29189, 'County'] = "ST. LOUIS COUNTY"

In [252]:
single_row.shape

(3150, 16)

In [253]:
corgis.shape

(3139, 28)

In [318]:
# Perform an inner merge on 'County' and 'State' columns
joined = pd.merge(corgis, single_row, on=["County", "State"], how='inner')
joined = joined.drop(columns=['county_name'])
joined

,County,State,Age.Percent 65 and Older,Education.Bachelor's Degree or Higher,Education.High School or Higher,Ethnicities.Hispanic or Latino,Ethnicities.White Alone,Ethnicities.White Alone\t not Hispanic or Latino,Housing.Households,Housing.Persons per Household,...,total_votes_2012,total_votes_2008,total_votes_2004,total_votes_2000,prop_favored_dem_2020,prop_favored_dem_2016,prop_favored_dem_2012,prop_favored_dem_2008,prop_favored_dem_2004,prop_favored_dem_2000
0,ABBEVILLE COUNTY,SC,22.4,15.6,81.7,1.6,70.2,68.9,9660,2.46,...,10671.0,11001.0,9925.0,8374.0,-0.330894,-0.280464,-0.134758,-0.151895,-0.105491,-0.081681
1,ACADIA PARISH,LA,15.8,13.3,79.0,2.8,79.7,77.3,22236,2.76,...,26835.0,26710.0,25230.0,23238.0,-0.603448,-0.566797,-0.498267,-0.456758,-0.283234,-0.211808
2,ACCOMACK COUNTY,VA,24.6,19.5,81.5,9.1,67.8,59.9,13438,2.35,...,16051.0,15623.0,13356.0,11925.0,-0.093975,-0.116513,-0.034764,-0.014466,-0.165319,-0.105660
3,ADA COUNTY,ID,14.9,38.5,95.2,8.5,91.9,84.3,173353,2.58,...,180546.0,179402.0,155002.0,123485.0,-0.039169,-0.092375,-0.113085,-0.061828,-0.233016,-0.278576
4,ADAIR COUNTY,IA,23.0,18.5,94.2,2.5,97.4,95.1,3200,2.17,...,3996.0,4053.0,4278.0,4123.0,-0.412736,-0.348465,-0.081081,-0.033555,-0.130435,-0.126607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3104,YUMA COUNTY,AZ,19.3,15.0,73.3,64.6,91.2,30.1,73098,2.79,...,41953.0,43615.0,38465.0,28652.0,-0.061877,-0.010564,-0.126165,-0.137980,-0.159938,-0.127495
3105,YUMA COUNTY,CO,18.7,21.8,88.6,25.1,96.5,72.9,4028,2.45,...,4577.0,4483.0,4559.0,4358.0,-0.666934,-0.652056,-0.546865,-0.483828,-0.524676,-0.475906
3106,ZAPATA COUNTY,TX,13.2,11.6,61.9,94.7,98.4,4.6,4503,3.17,...,3543.0,2866.0,2898.0,2616.0,-0.053433,0.329090,0.431837,0.355897,0.149758,0.261850
3107,ZAVALA COUNTY,TX,14.6,10.9,66.9,94.0,96.5,4.9,3571,3.33,...,3653.0,3876.0,3118.0,3391.0,0.313770,0.571849,0.675609,0.688080,0.498717,0.549985


In [319]:
joined['winner_2020'] = joined['prop_favored_dem_2020'].apply(lambda x: 'Democrat' if x > 0 else 'Republican')
joined

,County,State,Age.Percent 65 and Older,Education.Bachelor's Degree or Higher,Education.High School or Higher,Ethnicities.Hispanic or Latino,Ethnicities.White Alone,Ethnicities.White Alone\t not Hispanic or Latino,Housing.Households,Housing.Persons per Household,...,total_votes_2008,total_votes_2004,total_votes_2000,prop_favored_dem_2020,prop_favored_dem_2016,prop_favored_dem_2012,prop_favored_dem_2008,prop_favored_dem_2004,prop_favored_dem_2000,winner_2020
0,ABBEVILLE COUNTY,SC,22.4,15.6,81.7,1.6,70.2,68.9,9660,2.46,...,11001.0,9925.0,8374.0,-0.330894,-0.280464,-0.134758,-0.151895,-0.105491,-0.081681,Republican
1,ACADIA PARISH,LA,15.8,13.3,79.0,2.8,79.7,77.3,22236,2.76,...,26710.0,25230.0,23238.0,-0.603448,-0.566797,-0.498267,-0.456758,-0.283234,-0.211808,Republican
2,ACCOMACK COUNTY,VA,24.6,19.5,81.5,9.1,67.8,59.9,13438,2.35,...,15623.0,13356.0,11925.0,-0.093975,-0.116513,-0.034764,-0.014466,-0.165319,-0.105660,Republican
3,ADA COUNTY,ID,14.9,38.5,95.2,8.5,91.9,84.3,173353,2.58,...,179402.0,155002.0,123485.0,-0.039169,-0.092375,-0.113085,-0.061828,-0.233016,-0.278576,Republican
4,ADAIR COUNTY,IA,23.0,18.5,94.2,2.5,97.4,95.1,3200,2.17,...,4053.0,4278.0,4123.0,-0.412736,-0.348465,-0.081081,-0.033555,-0.130435,-0.126607,Republican
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3104,YUMA COUNTY,AZ,19.3,15.0,73.3,64.6,91.2,30.1,73098,2.79,...,43615.0,38465.0,28652.0,-0.061877,-0.010564,-0.126165,-0.137980,-0.159938,-0.127495,Republican
3105,YUMA COUNTY,CO,18.7,21.8,88.6,25.1,96.5,72.9,4028,2.45,...,4483.0,4559.0,4358.0,-0.666934,-0.652056,-0.546865,-0.483828,-0.524676,-0.475906,Republican
3106,ZAPATA COUNTY,TX,13.2,11.6,61.9,94.7,98.4,4.6,4503,3.17,...,2866.0,2898.0,2616.0,-0.053433,0.329090,0.431837,0.355897,0.149758,0.261850,Republican
3107,ZAVALA COUNTY,TX,14.6,10.9,66.9,94.0,96.5,4.9,3571,3.33,...,3876.0,3118.0,3391.0,0.313770,0.571849,0.675609,0.688080,0.498717,0.549985,Democrat


In [258]:
#joined.to_csv('/content/drive/My Drive/415/datasci 415/full_dataset_without_alaska.csv', index=False)